In [1]:
import numpy as np
import pandas as pd

In [2]:
#Mount the G Drive to access the data set
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
train_path = '/content/drive/MyDrive/Colab Notebooks/FP-DM/dataset/train_baseline_fix_index.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/FP-DM/dataset/test_baseline_fix_index.csv'
train_df = pd.read_csv(train_path, encoding= 'unicode_escape')
test_df = pd.read_csv(test_path, encoding= 'unicode_escape')
merged_df = train_df.append([test_df])

In [6]:
train_df

,Unnamed: 0,sentence #,word,tag,word_idx,tag_idx
0,0,1,Berikut,O,7983,0
1,1,1,adalah,O,2039,0
2,2,1,tujuh,O,4980,0
3,3,1,kota,O,41,0
4,4,1,di,O,4961,0
...,...,...,...,...,...,...
30243,30243,1464,kita,O,4705,0
30244,30244,1464,",",O,7275,0
30245,30245,1464,ucap,O,7791,0
30246,30246,1464,Fadli,B-PER,855,4


In [7]:
test_df

,Unnamed: 0,sentence #,word,tag,word_idx,tag_idx
0,0,1,Meski,O,4549,0
1,1,1,demikian,O,4487,0
2,2,1,",",O,7275,0
3,3,1,ia,O,5670,0
4,4,1,menegaskan,O,5148,0
...,...,...,...,...,...,...
18446,10583,509,keluarga,O,3747,0
18447,10584,509,-lah,O,2169,0
18448,10585,509,",",O,7275,0
18449,10586,509,ujarnya,O,3526,0


Siapkan data corpus dalam bentuk kalimat

In [8]:
data_merged_fillna = merged_df.fillna(method='ffill', axis=0)

data_merged_group = data_merged_fillna.groupby(['sentence #'],as_index=False)['word', 'tag', 'word_idx', 'tag_idx'].agg(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
data_merged_group

,sentence #,word,tag,word_idx,tag_idx
0,1,"[Berikut, adalah, tujuh, kota, di, Indonesia, ...","[O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O,...","[7983, 2039, 4980, 41, 4961, 1745, 5214, 5262,...","[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,"[Soal, calon, presiden, itu, urusan, nanti, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-P...","[3711, 8327, 3000, 5216, 4847, 865, 7275, 5214...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, ..."
2,3,"[Kalau, itu, tanya, PPP, saja, ,, ujar, Nasir,...","[O, O, O, B-ORG, O, O, O, B-PER, O, O, O, O, O...","[5035, 5216, 5722, 7446, 5829, 7275, 5990, 108...","[0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,"[Jokowi, bisa, mengatakan, mencabut, pengaduan...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[8234, 3106, 3220, 2834, 4775, 652, 3257, 2096...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,"[Akbar, menyebutkan, pemerintahan, Jokowi-JK, ...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[4662, 5284, 5706, 3052, 6288, 2366, 7513, 268...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
1459,1460,"[Dia, menggantikan, Ian, Campbell, ,, yang, di...","[O, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O...","[7122, 3901, 2733, 1094, 7275, 5214, 5571, 445...","[0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1460,1461,"[Kita, siapkan, strategi, baru, ,, bukan, gant...","[O, O, O, O, O, O, O, O, O, O, O]","[5581, 5206, 2338, 7187, 7275, 3022, 7872, 527...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1461,1462,"[September, 2007, PTBA, akan, mensosialisasika...","[O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O,...","[1820, 9, 8170, 607, 1804, 5973, 1372, 2594, 4...","[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1462,1463,"[Sedang, infeksi, virus, ebola, ini, berlangsu...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[2118, 8069, 1844, 4255, 452, 3596, 7595, 652,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [10]:
word_list = data_merged_group['word'].to_list()

Import Dictionary yang sudah dibangun pada saat preprocessing

In [15]:
# importing the module
import json
 
# Opening JSON file
def loadJSON(filename):
  data = {}
  with open(filename) as json_file:
      data = json.load(json_file)
  return data

In [16]:
idx2tag = loadJSON('/content/drive/MyDrive/Colab Notebooks/FP-DM/output/idx2tag.json')
tag2idx = loadJSON('/content/drive/MyDrive/Colab Notebooks/FP-DM/output/tag2idx.json')
token2idx = loadJSON('/content/drive/MyDrive/Colab Notebooks/FP-DM/output/token2idx.json')
idx2token = loadJSON('/content/drive/MyDrive/Colab Notebooks/FP-DM/output/idx2token.json')
print(idx2tag)

{'0': 'O', '1': 'I-LOC', '2': 'B-ORG', '3': 'I-PER', '4': 'B-PER', '5': 'I-ORG', '6': 'B-LOC'}


In [17]:
tag2idx = dict([key, int(value)] for key, value in tag2idx.items())
token2idx = dict([key, int(value)] for key, value in token2idx.items())
idx2tag = dict([int(key), value] for key, value in idx2tag.items())
idx2token = dict([int(key), value] for key, value in idx2token.items())
tag2idx

{'B-LOC': 6,
 'B-ORG': 2,
 'B-PER': 4,
 'I-LOC': 1,
 'I-ORG': 5,
 'I-PER': 3,
 'O': 0}

In [11]:
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 5.0 MB/s 


In [18]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus(token2idx) 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(word_list, window=5)

In [21]:
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=100, learning_rate=0.05)

glove.fit(corpus.matrix, epochs=25, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('/content/drive/MyDrive/Colab Notebooks/FP-DM/models/word_embeddings/glove5_100_25.txt')

Performing 25 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24


In [23]:
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=200, learning_rate=0.05)

glove.fit(corpus.matrix, epochs=25, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('/content/drive/MyDrive/Colab Notebooks/FP-DM/models/word_embeddings/glove5_200_25.txt')

Performing 25 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24


In [24]:
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=300, learning_rate=0.05)

glove.fit(corpus.matrix, epochs=25, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('/content/drive/MyDrive/Colab Notebooks/FP-DM/models/word_embeddings/glove5_300_25.txt')

Performing 25 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
